### Motivation
* This notebook is for exploring USA Census block group boundaries from a 2020 Census
We will try to see if we can use the existing feature layer created from Esri :  
Feature Layer: https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer  
Feature Layer Details: https://esri.maps.arcgis.com/home/item.html?id=2f5e592494d243b0aa5c253e75e792a4
* Using this layer, we would pick San Diego County FIPS data  
* Enrich the Dataframe with consumer spending and other variables

In [6]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geoenrichment import enrich, Country
#from feature_layer_utils import get_enrichment_variables
import itertools
import sys
sys.path.append('../../')
from utils import get_config

In [7]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

#### Selecting only San Diego County block groups from US block groups

In [8]:
fls = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroups/FeatureServer"

In [9]:
flc = FeatureLayerCollection(fls)

In [10]:
fl_url = flc.layers[0].url

In [11]:
us_block_grp_fl = FeatureLayer(fl_url)

Selecting FIPS='073' which is for San Diego County as our data

In [12]:
san_diego_county_fip = '073'
imperial_county_fip = '025'

In [13]:
san_diego_county_block_grp_fs = us_block_grp_fl.query(where=f"STATE_FIPS='06' AND COUNTY_FIPS='{san_diego_county_fip}'")

In [14]:
san_diego_county_block_grp_fs_sdf = san_diego_county_block_grp_fs.sdf
print(f"Shape: {san_diego_county_block_grp_fs_sdf.shape}")
san_diego_county_block_grp_fs_sdf.head(4)

Shape: (2057, 14)


,OBJECTID,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,BLOCKGROUP_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Area,Shape__Length,SHAPE
0,29002,CA,06,073,06073,000100,1,060730001001,1197,4788.0,0.25,0.000063,0.04273,"{""rings"": [[[-117.188940909064, 32.75880776425..."
1,29003,CA,06,073,06073,000100,2,060730001002,1711,5032.4,0.34,0.000084,0.052974,"{""rings"": [[[-117.187159908283, 32.75685976468..."
2,29004,CA,06,073,06073,000201,1,060730002011,877,4385.0,0.2,0.000049,0.036512,"{""rings"": [[[-117.168410904985, 32.75683776569..."
3,29005,CA,06,073,06073,000202,1,060730002021,1295,8633.3,0.15,0.000038,0.036458,"{""rings"": [[[-117.172296905513, 32.74893776444..."


#### Enrich the feature set for san diego county with variables

For enrichment, we can provide the feature set or the spatially enabled df.
Let us just pass down the sdf since we have visualized it already.

In [15]:
san_diego_block_groups_enriched_df = enrich(study_areas=san_diego_county_block_grp_fs_sdf,
                                           data_collections=['crime'],
                                           #analysis_variables=itertools.chain(*[market_potential_variables,demographics_variables,business_variables]),
                                           gis=gis,
                                       )
print(f"Shape: {san_diego_block_groups_enriched_df.shape}")
san_diego_block_groups_enriched_df.head(4)

Shape: (2057, 40)


,objectid,state_abbr,state_fips,county_fips,stcofips,tract_fips,blockgroup_fips,fips,totpop_cy,pop_sqmi,...,percagg,murdagg,rapeagg,robbagg,asstagg,procagg,burgagg,larcagg,mvehagg,SHAPE
0,29002,CA,06,073,06073,000100,1,060730001001,1197,4788.0,...,38368.0,20383.0,21582.0,58751.0,34771.0,92323.0,65945.0,88726.0,159467.0,"{""rings"": [[[-117.18894090906399, 32.758807764..."
1,29003,CA,06,073,06073,000100,2,060730001002,1711,5032.4,...,93060.0,54144.0,33840.0,147204.0,84600.0,170892.0,121824.0,170892.0,250416.0,"{""rings"": [[[-117.18715990828298, 32.756859764..."
2,29004,CA,06,073,06073,000201,1,060730002011,877,4385.0,...,108240.0,9922.0,82082.0,128986.0,106436.0,142516.0,82984.0,145222.0,221892.0,"{""rings"": [[[-117.168410904985, 32.75683776569..."
3,29005,CA,06,073,06073,000202,1,060730002021,1295,8633.3,...,32075.0,5132.0,89810.0,12830.0,30792.0,107772.0,102640.0,111621.0,94942.0,"{""rings"": [[[-117.17229690551298, 32.748937764..."


In [16]:
san_diego_block_groups_enriched_df.columns

Index(['objectid', 'state_abbr', 'state_fips', 'county_fips', 'stcofips',
       'tract_fips', 'blockgroup_fips', 'fips', 'totpop_cy', 'pop_sqmi',
       'sqmi', 'source_country', 'aggregation_method',
       'population_to_polygon_size_rating', 'apportionment_confidence',
       'has_data', 'totpop_cy', 'totpop_fy', 'popgrwcyfy', 'crmcytotc',
       'crmcyperc', 'crmcymurd', 'crmcyrape', 'crmcyrobb', 'crmcyasst',
       'crmcyproc', 'crmcyburg', 'crmcylarc', 'crmcymveh', 'totcagg',
       'percagg', 'murdagg', 'rapeagg', 'robbagg', 'asstagg', 'procagg',
       'burgagg', 'larcagg', 'mvehagg', 'SHAPE'],
      dtype='object')

In [18]:
san_diego_block_groups_enriched_df.to_csv('../resources/only_crime_enriched_san_diego_county_block_groups.csv', index=False)